## For Novice

In [ ]:
class AgenticRAG:
    def __init__(self, index):
        self.index = index
        self.hyde = HyDEQueryTransform(include_original=True)

    def reformulate_query(self, query):
        # General query reformulation for clarity and readability
        prompt = PromptTemplate(
            "You are a language expert. Reformulate the following query to make it clearer and easier to understand, "
            "while preserving its original meaning:\n\n"
            "Query: {query}\n\n"
            "Reformulated Query:"
        )

        response = llm.complete(prompt.format(query=query))
        return response.text.strip()

    def search_and_retrieve(self, query):
        # Use HyDE to generate a search query and retrieve relevant research data.
        query_engine = self.index.as_query_engine()
        hyde_query = self.hyde(query)
        return query_engine.query(hyde_query)
        # the goal is to improve the retrieval precision by instructing the HyDE to focus on key CNT params.
      #   prompt = PromptTemplate(
      #     "You are an expert in carbon nanotube (CNT) synthesis and properties. The following is a query related to CNTs:\n"
      #     "'{query}'\n"
      #     "Generate a hypothetical document embedding that captures the scientific context of this query. "
      #     "Ensure the embedding focuses on CNT-related factors such as diameter, chirality, electronic properties, functionalization techniques, and synthesis conditions. "
      #     "Use detailed, scientific phrasing to maximize the quality of the document retrieval."
      # )


    def analyze_results(self, results):
    # Analyze retrieved research results, focusing on relevant parameters and their impact on CNT synthesis.
        prompt = PromptTemplate(
            "You are explaining scientific results to a beginner:\n{results}\n"
            "Summarize the findings in a simple and clear way, using easy-to-understand language. "
            "Explain what the results mean in practical terms. Use examples or analogies to make complex concepts clearer. "
            "Avoid jargon and technical terms unless necessary, and explain any unfamiliar terms."
        )

        response = llm.complete(prompt.format(results=str(results)))
        return response.text

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def generate_final_answer(self, context, original_query):
        # Generate the final answer with specific focus on parameters like temperature, catalyst type, and growth rate.
          prompt = PromptTemplate(
            "You are answering a question for someone new to the field:\n"
            "Original Query: {original_query}\n"
            "Context: {context}\n"
            "Based on the research results, provide a clear and simple answer. "
            "Use beginner-friendly language, avoid complex terms, and explain any technical concepts. "
            "Include practical insights, real-world examples, and easy-to-follow explanations. "
            "If there are numerical values or data points, explain what they mean in plain terms."
        )

          response = llm.complete(prompt.format(context=context, original_query=original_query))
          return response.text

    def process_query(self, initial_query):
        # Reformulate, retrieve, and generate the final answer based on the user query and research context.
        refined_query = self.reformulate_query(initial_query)
        print(f"Refined Query: {refined_query}")

        results = self.search_and_retrieve(refined_query)
        print(f"Retrieved Results: {results}")

        final_answer = self.generate_final_answer(results, initial_query)
        print(f"Final Answer: {final_answer}")

        return final_answer


## For Advanced

In [ ]:
class AgenticRAG:
    def __init__(self, index):
        self.index = index
        self.hyde = HyDEQueryTransform(include_original=True)

    def reformulate_query(self, query):
        # Reformulate the user's question to focus on key factors related to CNT synthesis.
        prompt = PromptTemplate(
            "The user's query is related to carbon nanotube (CNT) synthesis: '{query}'\n"
            "Reformulate this query to emphasize critical experimental conditions, including exact temperature settings, catalyst composition, and thickness, as well as detailed growth rate trends. Focus on extracting nuanced insights about stability, growth rates, and structural quality during synthesis."
            "Reformulated query:"
        )
        response = llm.complete(prompt.format(query=query))
        return response.text.strip()

    def search_and_retrieve(self, query):
        # Use HyDE to generate a search query and retrieve relevant research data.
        query_engine = self.index.as_query_engine()
        hyde_query = self.hyde(query)
        return query_engine.query(hyde_query)

    def analyze_results(self, results):
        # Analyze retrieved research results, focusing on relevant parameters and their impact on CNT synthesis.
        prompt = PromptTemplate(
            "Analyze the following advanced research results on CNT synthesis:\n{results}\n"
            "- Extract key findings about the influence of factors like temperature (e.g., 604°C), catalyst type (e.g., Fe or Aluminum Oxide), and thickness (e.g., 0.61 µm) on growth rates, stability, and maximum height."
            "- Include advanced observations such as growth stabilization times (e.g., 1118 seconds), numerical trends, and any anomalies in the data."
            "- Suggest high-level optimizations or experimental modifications to refine CNT synthesis, ensuring a focus on quality and efficiency."
            "Analysis:"
        )

        response = llm.complete(prompt.format(results=str(results)))
        return response.text

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def generate_final_answer(self, context, original_query):
        # Generate the final answer with specific focus on parameters like temperature, catalyst type, and growth rate.
        prompt = PromptTemplate(
            "The user seeks a detailed understanding of CNT synthesis and its influencing parameters with the following specifics:\n"
            "- Temperature: {temperature}\n"
            "- Catalyst type: {catalyst_type}\n"
            "- Catalyst thickness: {thickness}\n"
            "- Observed growth rate trends and final height: {context}\n\n"
            "As an experienced researcher, delve deeply into the analysis and provide:\n"
            "1. A comprehensive technical explanation of the interplay between these parameters, emphasizing the underlying mechanisms that affect CNT growth stability, maximum height, and overall quality.\n"
            "2. In-depth numerical insights, including specific trends and their theoretical or experimental justification (e.g., correlation between catalyst thickness and growth kinetics, or optimal temperature ranges for enhanced structural integrity).\n"
            "3. For theory-based queries, discuss underlying mechanisms, material properties, and their impact on synthesis outcomes, referencing established scientific principles and models.\n"
            "4. For practical queries, offer advanced experimental adjustments to refine synthesis, supported by examples from recent literature and contextual insights from the provided data.\n\n"
            "Ensure the response is exhaustive, integrates theoretical concepts with practical implications, and highlights both foundational knowledge and state-of-the-art advancements in CNT research.\n\n"
            "Answer:"
        )

        response = llm.complete(prompt.format(context=context, original_query=original_query))
        return response.text

    def process_query(self, initial_query):
        # Reformulate, retrieve, and generate the final answer based on the user query and research context.
        refined_query = self.reformulate_query(initial_query)
        print(f"Refined Query: {refined_query}")

        results = self.search_and_retrieve(refined_query)
        print(f"Retrieved Results: {results}")

        final_answer = self.generate_final_answer(results, initial_query)
        print(f"Final Answer: {final_answer}")

        return final_answer


## **Without Reformulation**

In [ ]:
class AgenticRAG:
    def __init__(self, index):
        self.index = index
        self.hyde = HyDEQueryTransform(include_original=True)

    def search_and_retrieve(self, query):
        # Use HyDE to generate a search query and retrieve relevant research data.
        query_engine = self.index.as_query_engine()
        hyde_query = self.hyde(query)
        return query_engine.query(hyde_query)

        # Improve retrieval precision by instructing HyDE to focus on key CNT parameters.
        prompt = PromptTemplate(
            "You are an expert in carbon nanotube (CNT) synthesis and properties. The following is a query related to CNTs:\n"
            "'{query}'\n"
            "Generate a hypothetical document embedding that captures the scientific context of this query. "
            "Ensure the embedding focuses on CNT-related factors such as diameter, chirality, electronic properties, "
            "functionalization techniques, and synthesis conditions. "
            "Use detailed, scientific phrasing to maximize the quality of the document retrieval."
        )

    def analyze_results(self, results):
        # Analyze retrieved research results, focusing on relevant parameters and their impact on CNT synthesis.
        prompt = PromptTemplate(
            "You are analyzing scientific results related to CNT synthesis:\n{results}\n"
            "Summarize the findings in a clear and scientific manner. "
            "Focus on specific factors such as temperature, catalyst type, gas flow rate, and growth rate. "
            "Highlight key observations, including numerical values (e.g., temperature ranges, CNT diameter, catalyst thickness) "
            "and their influence on CNT quality, structural integrity, and electronic properties. "
            "Provide practical insights for optimizing CNT synthesis, including specific experimental adjustments or recommendations "
            "based on the research findings. "
            "Make the summary clear and actionable for both domain experts and new researchers."
        )

        response = llm.complete(prompt.format(results=str(results)))
        return response.text

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def generate_final_answer(self, context, original_query):
        # Generate the final answer with specific focus on parameters like temperature, catalyst type, and growth rate.
        prompt = PromptTemplate(
            "You are generating a detailed answer to the following query about carbon nanotubes (CNTs):\n"
            "Original Query: {original_query}\n"
            "Context: {context}\n"
            "Based on the research results, provide a comprehensive and scientifically accurate answer. "
            "Explain how key parameters (temperature, catalyst type, synthesis method, and growth rate) influence CNT growth, structure, and properties. "
            "Include specific numerical values (e.g., temperature ranges, catalyst particle sizes, gas flow rates) and practical recommendations for optimizing CNT synthesis. "
            "Ensure the answer is clear, concise, and informative, suitable for both new researchers and domain experts."
        )

        response = llm.complete(prompt.format(context=context, original_query=original_query))
        return response.text

    def process_query(self, initial_query):
        # Retrieve and generate the final answer based on the user query and research context.
        results = self.search_and_retrieve(initial_query)
        print(f"Retrieved Results: {results}")

        # final_answer = self.generate_final_answer(results, initial_query)
        # print(f"Final Answer: {final_answer}")
        return results

        # return final_answer